In [ ]:
# !pip install imageio

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import imageio.v2 as imageio
import os

In [ ]:
# Load shapefiles of all seven provinces
province_shapes = {}
for i in range(1, 8):
    province_shapes[f'Pr{i}'] = gpd.read_file(f'/home/rabina/Projects/rabina/MAP VISUALIZATION/shapefiles/Pr{i}.shp')
    
# Concatenate all provinces into a single GeoDataFrame
all_provinces = pd.concat(province_shapes.values(), ignore_index=True)

all_provinces.rename(columns={'PROVINCE': 'Province'}, inplace=True)

hazard_data=pd.read_csv(r'/home/rabina/Projects/rabina/MAP VISUALIZATION/provine_wise_hazard_combined_sheet.csv')

hazard_data_columns=hazard_data.columns.to_list()
hazard_data_columns.remove('Year')
hazard_data_columns.remove('Province')

hazard_data['Year']=pd.to_datetime(hazard_data['Year'],format='%Y')
hazard_data['Year'] = hazard_data['Year'].dt.strftime('%Y')

In [ ]:
# Iterate over each year
global_min=0

# hazard_data_columns=['Flood']

max_limits={'Flood':131, 'Landslide':170, 'Storm':17, 'Earthquake':77, 'Heavy Rainfall':98, 'Cold Wave':46, 'Animal Incidents':160, 'Forest Fire':315, 'Avalanche':3, 'Wind Storm':69, 'Fire':313, 'Snow Storm':2, 'Thunderbolt':93, 'Epidemic':17}


for hazard_name in hazard_data_columns:
    print(f"******{hazard_name}************")
    frames=[]
    global_max=max_limits[hazard_name]
    try:
        for year in range(2011, 2025):
            # Create a figure and axis
            fig, ax = plt.subplots(figsize=(10, 10))

            # Filter hazard data for the current year
            hazard_data_year = hazard_data[hazard_data['Year'] == str(year)].astype(str)

            # Merge hazard data with all provinces
            all_provinces['Province'] = all_provinces['Province'].astype(str)
            hazard_data['Province'] = hazard_data['Province'].astype(str)
            
            merged = all_provinces.merge(hazard_data_year, on='Province')
            merged[hazard_name] = pd.to_numeric(merged[hazard_name], errors='coerce')
            
            # Plot all provinces together, coloring based on the number of hazard incidents for the current year
            merged.plot(column=hazard_name, cmap='Oranges', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True,vmin=global_min, vmax=global_max)

            # Set plot title and labels
            ax.set_title(f'{hazard_name} Incidents in Nepal - Year {year}')
            ax.set_xlabel('Longitude')
            ax.set_ylabel('Latitude')
            
            # Save the plot as an image
            temp_image_path=f'/home/rabina/Projects/rabina/MAP VISUALIZATION/temp_output/nepal_{hazard_name}_{year}.png'
            plt.savefig(temp_image_path)
            plt.show()
            
            # Append the file path to the list of frames
            frames.append(temp_image_path)
            # Close the plot to avoid memory leaks
            plt.close()

        # Create the animation
        animation_path = f"./animated_output/{hazard_name}.gif"
        with imageio.get_writer(animation_path, mode='I', duration=6,fps=1) as writer:
            for temp_image_path in frames:
                image = imageio.imread(temp_image_path)
                writer.append_data(image)
                
        # Clean up temporary image files
        for temp_image_path in frames:
            os.remove(temp_image_path)
        print("Animation creation completed.")
    except Exception as e:
        print(str(e))